In [96]:
include("Lorenz.jl")
include("UKI.jl")
include("uki_v4_paper_functions.jl")
Threads.nthreads()
using CSV, DataFrames, JLD2

In [ ]:
data_save_dir = "saved_data/ergodic_tests"
figure_save_dir = "saved_figures/ergodic_tests"

# Test Ergodic Parameter Configurations
from_save = true

uki_iters = 150
timestep = .1

t_ends = [10, 50, 100, 500, 1000, 1500]
sigma_eta_mags = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
noise_mags = [.01, .05, .1]
C_0_mags = [1, .1, .01, .001, 1e-4]

results_df = DataFrame(t_end = Float64[], sigma_eta_mag = Float64[], noise_mag = Float64[], C_0_mag = Float64[],
    threshold_01 = Float64[], threshold_015 = Float64[], threshold_03 = Float64[], threshold_01_cost = Float64[],
    threshold_015_cost = Float64[], threshold_03_cost = Float64[], threshold_03_reached = Bool[])

for t_end in t_ends
    for sigma_eta_mag in sigma_eta_mags
        for noise_mag in noise_mags
            for C_0_mag in C_0_mags
                
                # Convert t_end, timestep, noise, sigma_eta_mag and C_0_mag to string with scientific notation
                parameter_string = @sprintf("%.0e", t_end) * "_" * @sprintf("%.0e", timestep) * "_" * @sprintf("%.0e", noise_mag) * "_" * @sprintf("%.0e", sigma_eta_mag) * "_" * @sprintf("%.0e", C_0_mag)
                data_filename = joinpath(data_save_dir, "ergodic_params_$(parameter_string).jld2")
                plot_filename = joinpath(figure_save_dir, "ergodic_$(parameter_string).png")
                loss_plot_filename = joinpath(figure_save_dir, "ergodic_loss_$(parameter_string).png")
                
                string_t_end = string(t_end)
                string_lambda_v = string(floor(Int, log10(abs(sigma_eta_mag))))
                string_lambda_w = string(floor(Int, log10(abs(C_0_mag))))
                string_noise = string(@sprintf("%.0f%%", noise_mag * 100))
                
                title = "Lorenz '96 Ergodic UKI Calibration\n" * 
                    L"\Delta t=" * string(t_end) * ", " *
                    L" \mathrm{Noise} = " * string_noise * ", " * 
                    L"\lambda_v = " * string_lambda_v * ", " *
                    L"\lambda_w = " * string_lambda_w
                
#                 title = "Lorenz '96 Ergodic UKI Calibration\n" * 
#                 L"\Delta t=$(t_end), Noise = $(string_noise), \lambda_v = $(string_lambda_v), \lambda_w = $(string_lambda_w)"

                toss = Int(t_end/10)
                if from_save && isfile(data_filename)
                    _, stable_iters, costs, reached = plot_ergodic_test_from_file(data_filename, plot_filename, loss_plot_filename, title, t_end, show_plot=true);
                    
                else
                    _, stable_iters, costs, reached = ergodic_test(t_end, timestep, sigma_eta_mag, C_0_mag, uki_iters,
                        prior_theta = [0.1, 5, 2, 7], obs_noise = noise_mag, toss=toss, plot_progress=false, data_filename=data_filename, plot_filename=plot_filename, loss_plot_filename=loss_plot_filename, show_plot=false);
                end
                
                df_row = [t_end, sigma_eta_mag, noise_mag, C_0_mag, stable_iters..., costs..., reached]
                push!(results_df, df_row)
                
            end
        end
    end
end

In [3]:
for (colname, column) in pairs(eachcol(results_df))
    if eltype(column) <: Float64
        replace!(results_df[!, colname], -1.0 => Inf)
    end
end

In [ ]:
candidate_df = filter(row -> row.threshold_03_reached == true, results_df)
candidate_df_unique = select(candidate_df, [:t_end, :sigma_eta_mag, :C_0_mag])
candidate_df_unique = unique(candidate_df_unique)
display(candidate_df_unique)
data_save_dir = "saved_data/ergodic_tests"
filename = joinpath(data_save_dir, "uki_ergodic_candidate_params.jld2")
save_data(candidate_df_unique, filename)

In [ ]:
data_save_dir = "saved_data/ergodic_tests"
filename = joinpath(data_save_dir, "uki_ergodic_candidate_params.jld2")
data = load(filename)["data"]

In [ ]:
filtered_df = filter(row -> row.threshold_03_cost != Inf, results_df)
# sorted_filtered_df = sort(filtered_df, :threshold_0_cost)
display(filtered_df)
subset_df = select(filtered_df, [:t_end, :sigma_eta_mag, :C_0_mag])
unique_subset = unique(subset_df)

In [ ]:
uki_iters = 150
timestep = .1

expanded_best_results_repeats_df = DataFrame(t_end = Float64[], sigma_eta_mag = Float64[], noise_mag = Float64[], C_0_mag = Float64[],
    threshold_01 = Float64[], threshold_015 = Float64[], threshold_03 = Float64[], threshold_01_cost = Float64[],
    threshold_015_cost = Float64[], threshold_03_cost = Float64[], repeat=Int64[])
repeat_count = 20
noise_mags = [.01, .05, .1]

for row in eachrow(unique_subset)
    for repeat in 1:repeat_count
        for noise_mag in noise_mags
            t_end = row.t_end
            sigma_eta_mag = row.sigma_eta_mag
            C_0_mag = row.C_0_mag
            toss = Int(t_end/10)

            _, stable_iters, costs = ergodic_test(t_end, timestep, sigma_eta_mag, C_0_mag, uki_iters,
                                prior_theta = [0.1, 5, 2, 7], obs_noise = noise_mag, toss=toss, plot_progress=false,);

            df_row = [t_end, sigma_eta_mag, noise_mag, C_0_mag, stable_iters..., costs..., repeat]
            push!(expanded_best_results_repeats_df, df_row)
        end
    end
end

In [ ]:
data_save_dir = "saved_data/ergodic_tests"
filename = joinpath(data_save_dir, "uki_best_param_ergodic_repeat_results_expanded.jld")
save_data(expanded_best_results_repeats_df, filename)

In [ ]:
# Collect and Repeat the minimum cost scenarios
sort_columns = ["threshold_01_cost", "threshold_015_cost", "threshold_03_cost"]
noise_mags = [0.01, 0.05, 0.1]

min_params_df = similar(results_df, 0)

for sort_column in sort_columns
    for noise_mag in noise_mags
        filtered_df = filter(row -> row.noise_mag == noise_mag, results_df)
        sorted_filtered_df = sort(filtered_df, sort_column)
        push!(min_params_df, sorted_filtered_df[1, :])
    end
end

min_params_df = unique(min_params_df)

In [ ]:
for (colname, column) in pairs(eachcol(best_results_repeats_df))
    if eltype(column) <: Float64
        replace!(best_results_repeats_df[!, colname], -1.0 => Inf)
    end
end
# filtered_df = filter(row -> row.noise_mag == 0.1, best_results_repeats_df)
filtered_df = filter(row -> row.threshold_03_cost == Inf, best_results_repeats_df)
sort(filtered_df, :threshold_03_cost)

In [ ]:
df = analyze_ergodic_results(filtered_df, success_threshold = .01)
display(sort(df[df[!, "noise_mag"] .== .1, :], "mean_cost"))
display(sort(df[df[!, "noise_mag"] .== .05, :], "mean_cost"))
display(sort(df[df[!, "noise_mag"] .== .01, :], "mean_cost"))

In [ ]:
# UF
include("Lorenz.jl")
include("UKI.jl")
include("uki_v4_paper_functions.jl")
using CSV, DataFrames

from_save = true

prior_theta = [0.1, 5, 2, 7]
timesteps = [.1]
t_end = 1000
batch_sizes = [10, 50, 100, 200, 400]
uki_iters = 500
sigma_eta_mags = [1e-3, 1e-4, 1e-5, 1e-6]
C_0_mags = [.1, .01, 1e-3]
noise_mags = [.01, .05, .1]

figure_save_dir = "l96_spring_paper_saved/saved_figures/batch_tests"
data_save_dir = "l96_spring_paper_saved/saved_data/batch_tests"

results_df = DataFrame(t_end = Float64[], batch_size = Float64[], sigma_eta_mag = Float64[], noise_mag = Float64[], C_0_mag = Float64[],
    threshold_01 = Float64[], threshold_015 = Float64[], threshold_03 = Float64[], threshold_01_cost = Float64[],
    threshold_015_cost = Float64[], threshold_03_cost = Float64[], threshold_03_reached = Bool[])

learning_rate_tuple = nothing

for timestep in timesteps
    for batch_size in batch_sizes
        for sigma_eta_mag in sigma_eta_mags
            for C_0_mag in C_0_mags
                for noise_mag in noise_mags
                    parameter_string = "new_noise_"*@sprintf("%.0e", t_end) * "_" * @sprintf("%.0e", timestep) * "_" * @sprintf("%.0e", noise_mag) * "_" * @sprintf("%.0e", sigma_eta_mag) * "_" * @sprintf("%.0e", C_0_mag)* "_" * @sprintf("%.0e", batch_size)
                    plot_filename = joinpath(figure_save_dir, "batch_test_$(parameter_string).png")
#                     plot_filename=nothing
                    
                    data_filename = joinpath(data_save_dir, "batch_test_$(parameter_string).jld2")
                    
                    string_t_end = string(t_end)
                    string_lambda_v = string(floor(Int, log10(abs(sigma_eta_mag))))
                    string_lambda_w = string(floor(Int, log10(abs(C_0_mag))))
                    string_noise = string(@sprintf("%.0f%%", noise_mag * 100))

                    title = "Lorenz '96 UF-UKI Calibration\n" * 
                        L"B=" * string(batch_size) * ", " *
                        L" \mathrm{Noise} = " * string_noise * ", " * 
                        L"\lambda_v = " * string_lambda_v * ", " *
                        L"\lambda_w = " * string_lambda_w
                    
#                     title = "Batch L96 Calibration\n" * "Batch Size=" * string(batch_size) *
#                         "T=" * string(t_end) * L", O(\textrm{Noise}) = " * string(@sprintf("%.2e", noise_mag)) * L", " *
#                         L"O(\sigma_\eta) = " * string(@sprintf("%.2e", sigma_eta_mag)) * L", " *
#                         L"O(C_0) = " * string(@sprintf("%.2e", C_0_mag))

                    if from_save && isfile(data_filename)
                        println("Combination exists")
                        uki_params, stable_iters, costs, reached = batch_test(t_end, timestep, batch_size, sigma_eta_mag, C_0_mag, uki_iters, prior_theta = prior_theta, obs_noise = noise_mag, plot_filename=plot_filename, data_filename=data_filename, plot_from_data=true, show_plot=true, title_str=title);

                    else
                        uki_params, stable_iters, costs, reached = batch_test(t_end, timestep, batch_size, sigma_eta_mag, C_0_mag, uki_iters, prior_theta = prior_theta, obs_noise = noise_mag, plot_filename=plot_filename, data_filename=data_filename, show_plot=false);
                    end
                
                    df_row = [t_end, batch_size, sigma_eta_mag, noise_mag, C_0_mag, stable_iters..., costs..., reached]
                    push!(results_df, df_row)
                
                end
            end
        end
    end
end

In [ ]:
for (colname, column) in pairs(eachcol(results_df))
    if eltype(column) <: Float64
        replace!(results_df[!, colname], -1.0 => Inf)
    end
end

In [ ]:
candidate_df = filter(row -> row.threshold_03_reached == true, results_df)
candidate_df_unique = select(candidate_df, [:batch_size, :sigma_eta_mag, :C_0_mag])
candidate_df_unique = unique(candidate_df_unique)
display(candidate_df_unique)
data_save_dir = "saved_data/batch_tests"
filename = joinpath(data_save_dir, "uki_batch_candidate_params.jld2")
save_data(candidate_df_unique, filename)

In [ ]:
data_save_dir = "saved_data/batch_tests"
filename = joinpath(data_save_dir, "uki_batch_candidate_params.jld2")
data = load(filename)["data"]

In [ ]:
# Collect and Repeat the minimum cost scenarios
sort_columns = ["threshold_01_cost", "threshold_015_cost", "threshold_03_cost"]
noise_mags = [0.01, 0.05, 0.1]

min_params_df = similar(results_df, 0)

for sort_column in sort_columns
    for noise_mag in noise_mags
        filtered_df = filter(row -> row.noise_mag == noise_mag, results_df)
        sorted_filtered_df = sort(filtered_df, sort_column)
        push!(min_params_df, sorted_filtered_df[1, :])
    end
end

min_params_df = unique(min_params_df)

In [ ]:
uki_iters = 500
timestep = .1

expanded_best_results_repeats_df = DataFrame(t_end = Float64[], batch_size = Float64[], sigma_eta_mag = Float64[], noise_mag = Float64[], C_0_mag = Float64[],
    threshold_01 = Float64[], threshold_015 = Float64[], threshold_03 = Float64[], threshold_01_cost = Float64[],
    threshold_015_cost = Float64[], threshold_03_cost = Float64[], repeat=Int64[])
repeat_count = 20
noise_mags = [.01, .05, .1]

for row in eachrow(unique_subset)
    for repeat in 1:repeat_count
        for noise_mag in noise_mags
            t_end = 1000
            sigma_eta_mag = row.sigma_eta_mag
            C_0_mag = row.C_0_mag
            batch_size = Int64(row.batch_size)       

            _, stable_iters, costs = batch_test(t_end, timestep, batch_size, sigma_eta_mag, C_0_mag, uki_iters,
                prior_theta = [0.1, 5, 2, 7], obs_noise = noise_mag, show_plot=false);

            df_row = [t_end, batch_size, sigma_eta_mag, noise_mag, C_0_mag, stable_iters..., costs..., repeat]
            push!(expanded_best_results_repeats_df, df_row)
        end
    end
end

In [ ]:
data_save_dir = "saved_data/batch_tests"
filename = joinpath(data_save_dir, "uki_best_param_batch_repeat_results_expanded.jld")
save_data(expanded_best_results_repeats_df, filename)

In [ ]:
unique_rows_df = unique(expanded_best_results_repeats_df)
df = analyze_batch_results(unique_rows_df)
display(sort(df[df[!, "noise_mag"] .== .1, :], "mean_cost"))
display(sort(df[df[!, "noise_mag"] .== .05, :], "mean_cost"))
display(sort(df[df[!, "noise_mag"] .== .01, :], "mean_cost"))

In [ ]:
# sample L96 plots
theta = [1.0, 10.0, log(10), 10.0]

timesteps = [.01, .1, 1, 5]
noise_mags = [0, 1e-2, 5e-2, 1e-1]

previous_output=nothing

for timestep in timesteps
    for noise_mag in noise_mags
        
        t_end = 100
        plot_length = timestep * 10

        parameter_string = @sprintf("%.0e", t_end) * "_" * @sprintf("%.0e", timestep) * "_" * @sprintf("%.0e", noise_mag)
        figure_filename = nothing
#         figure_filename = joinpath(figure_save_dir, "noisy_trajectory_comparison_new_noise_$(parameter_string).png")
        
        if isnothing(previous_output)
            previous_output = plot_noisy_L96_trajectory(t_end, plot_length, timestep, theta, noise_mag, filename=figure_filename, noise_on_both=true)
        else
            plot_noisy_L96_trajectory(t_end, plot_length, timestep, theta, noise_mag, filename=figure_filename, noise_on_both=true, previous_output=previous_output)
        end
    end
end